In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix


 I've saved the under_sampled_train_data and test_data_encoded1 files from the R code on my computer with under_sampled_train_data_encoded2 and test_data_encoded1 as name

In [68]:
df_train = pd.read_csv(r"C:\Users\User\Documents\Horse Racing\under_sampled_train_data_encoded2.csv")
#df_train = pd.read_csv(r"C:\Users\User\Documents\Horse Racing\under_sampled_train_data_encoded_interact.csv")
#df_train = pd.read_csv(r"C:\Users\User\Documents\Horse Racing\train_data_encoded1_without_over_under.csv")
df_test = pd.read_csv(r"C:\Users\User\Documents\Horse Racing\test_data_encoded2.csv")

In [3]:
RaceID_To_Norm = pd.read_csv(r"C:\Users\User\Documents\Horse Racing\RaceID_To_Norm.csv")
RaceID_To_Norm = RaceID_To_Norm.drop(['Unnamed: 0'], axis=1)

In [69]:
y_train = df_train["RaceOutcome"]
x_train = df_train.drop(['RaceOutcome','Unnamed: 0','HorseID','DamID'], axis=1)
y_test = df_test["RaceOutcome"]
x_test = df_test.drop(['RaceOutcome','Unnamed: 0','HorseID','DamID'], axis=1)

In [6]:
from keras import optimizers

In [7]:
from tensorflow.keras.callbacks import Callback

class CustomEarlyStop(Callback):
    def __init__(self, monitor='val_acc', min_delta=0.5, patience=0):
        super(CustomEarlyStop, self).__init__()
        self.monitor = monitor
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.best = 0.95  # Set the threshold to 0.8 for val_acc

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            return

        if current > self.best:
            print(f"Stopping training as {self.monitor} exceeded 0.9!")
            self.model.stop_training = True

# Usage:
custom_early_stop = CustomEarlyStop(monitor='val_acc', min_delta=0.0)  # Set min_delta to 0.0 for this scenario

# Model fitting
#model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[custom_early_stop])


In [8]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

tf.random.set_seed(42)

# Adjusted hyperparameters
hidden_units = 256 # Increased units for more complexity
learning_rate = 0.03  # Further reduced learning rate
no_epochs = 70  # Increased epochs for more training

model = Sequential()

model.add(Dense(hidden_units, input_dim=92, activation='relu'))
model.add(BatchNormalization())  # Adding batch normalization for stability
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(0.3))  # Increasing dropout for regularization
model.add(Dense(1, activation='sigmoid'))

# Using Adam optimizer with a low learning rate
adam = optimizers.Adam(learning_rate)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])

es = EarlyStopping(monitor='val_acc', 
                                   mode='max', # don't minimize the accuracy!
                                   patience=25,
                                   restore_best_weights=True)

# Training the model
model.fit(x_train, y_train, epochs=no_epochs, batch_size=128, verbose=2,validation_data=(x_test, y_test),callbacks=[es]) #


Epoch 1/70
2150/2150 - 12s - loss: 0.5619 - acc: 0.7285 - val_loss: 0.5233 - val_acc: 0.7856 - 12s/epoch - 5ms/step
Epoch 2/70
2150/2150 - 10s - loss: 0.5536 - acc: 0.7321 - val_loss: 0.4791 - val_acc: 0.7877 - 10s/epoch - 5ms/step
Epoch 3/70
2150/2150 - 9s - loss: 0.5511 - acc: 0.7337 - val_loss: 0.6145 - val_acc: 0.7318 - 9s/epoch - 4ms/step
Epoch 4/70
2150/2150 - 9s - loss: 0.5523 - acc: 0.7325 - val_loss: 0.5240 - val_acc: 0.7775 - 9s/epoch - 4ms/step
Epoch 5/70
2150/2150 - 10s - loss: 0.5536 - acc: 0.7313 - val_loss: 0.5615 - val_acc: 0.7519 - 10s/epoch - 5ms/step
Epoch 6/70
2150/2150 - 11s - loss: 0.5529 - acc: 0.7323 - val_loss: 0.4767 - val_acc: 0.8134 - 11s/epoch - 5ms/step
Epoch 7/70
2150/2150 - 10s - loss: 0.5506 - acc: 0.7346 - val_loss: 0.5424 - val_acc: 0.7835 - 10s/epoch - 5ms/step
Epoch 8/70
2150/2150 - 10s - loss: 0.5531 - acc: 0.7323 - val_loss: 0.4999 - val_acc: 0.7848 - 10s/epoch - 5ms/step
Epoch 9/70
2150/2150 - 10s - loss: 0.5520 - acc: 0.7322 - val_loss: 0.5312 -

## Add the predict_proba column from test_set to the RaceId_to_Norm

In [59]:
predicted_=model.predict(x_test)

predicted_series = pd.Series(predicted_.flatten())  
RaceID_To_Norm['predicted_probabilities'] = predicted_series
RaceID_To_Norm['RaceOutcome'] = y_test


872/872 [==============================] - 2s 2ms/step


In [61]:
import pandas as pd
import numpy as np

# Sort the data by the original index to retain the initial order after grouping
RaceID_To_Norm = RaceID_To_Norm.sort_index()

# Grouping data by RaceID
grouped = RaceID_To_Norm.groupby('RaceID')

# Define a function to normalize probabilities using exponential function
def normalize_probs(group):
    exp_probs = np.exp(group['predicted_probabilities'])
    sum_exp_probs = exp_probs.sum()
    group['win_probability'] = exp_probs / sum_exp_probs
    
    # Adjusting for precision to ensure the sum is exactly 1
    group['win_probability'] /= group['win_probability'].sum()
    
    return group

# Applying the normalization function to each group (race)
Normalized_prob = grouped.apply(normalize_probs).reset_index(drop=True)


In [66]:
# Save the DataFrame to a CSV file
Normalized_prob.to_csv(r'C:\Users\User\Documents\Horse Racing\python produced file\win_probability.parquet', index=False)
